In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.GANMF.GANMF import GANMF

model = GANMF

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])

2023-12-09 11:37:04.685606: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 11:37:04.685660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 11:37:04.687810: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 11:37:04.696555: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 11:37:05.605865: W tensorflow/compiler/tf2

EvaluatorHoldout: Ignoring 2595 (20.5%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Ignoring 2195 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
recommender = model(URM_train)
# recommender.fit(**trial.params, workers=6)
recommender.fit(validation_evaluator=evaluator_validation)

/home/federico/Documents/projects/rec-sys/RecSys-2023-polimi/Recommenders/GANMF/GANMF.py:64: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  encoding = tf.compat.v1.layers.dense(input_data, units=emb_dim, kernel_initializer=glorot_uniform,
/home/federico/Documents/projects/rec-sys/RecSys-2023-polimi/Recommenders/GANMF/GANMF.py:66: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  decoding = tf.compat.v1.layers.dense(encoding, units=self.num_items, kernel_initializer=glorot_uniform,
2023-12-09 10:48:14.377086: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-09 1

Instructions for updating:
Use `tf.global_variables_initializer` instead.


2023-12-09 10:48:14.633860: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-09 10:48:14.634164: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-09 10:48:14.634343: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

KeyboardInterrupt: 

In [4]:
evaluator_test.evaluateRecommender(recommender)

EvaluatorHoldout: Ignoring 2172 (17.2%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10466 (100.0%) in 6.77 sec. Users per second: 1547


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.034903                 0.060857  0.051631  0.014789    0.025143   
 
              MRR      NDCG       F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.107267  0.052758  0.04165  0.257214      0.125521  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.828137          0.213008    0.828137       0.005405   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            7.034099                   0.985282             0.015711   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                      

In [11]:
import optuna as op
def objective(trial):
    trial.suggest_int("num_factors", 1, 250)
    trial.suggest_int("batch_size", 64, 1024)
    trial.suggest_int("m", 1, 10)
    trial.suggest_float("d_lr", 1e-4, 1e-2, log=True)
    trial.suggest_float("g_lr", 1e-4, 1e-2, log=True)
    trial.suggest_float("d_reg", 1e-6, 1e-4, log=True)
    trial.suggest_float("recon_coefficient", 1e-2, 0.5)
    
    recommender = model(URM_train)
    recommender.fit(**trial.params, epochs=30, validation_evaluator=evaluator_validation, freq=2, after=2)
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "num_factors":245,
    "batch_size":747,
    "m":6,
    "d_lr":0.0016329762690425955,
    "g_lr":0.00020240992201659615,
    "d_reg":4.7660661730127275e-06
}

study_name = "GANMF-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
# study.enqueue_trial(best_params)
study.optimize(objective, n_trials=300)

In [15]:
study.best_params = best_params

NameError: name 'best_params' is not defined

In [3]:
best_params = {
    "num_factors":245,
    "batch_size":747,
    "m":6,
    "d_lr":0.0016329762690425955,
    "g_lr":0.00020240992201659615,
    "d_reg":4.7660661730127275e-06
}
# final = model(URM_train)
# final.fit(**best_params,validation_evaluator=evaluator_validation, freq=4, after=25)

final = model(URM_train_validation)
final.fit(**best_params, epochs=50)

/home/federico/Documents/projects/rec-sys/RecSys-2023-polimi/Recommenders/GANMF/GANMF.py:65: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  encoding = tf.compat.v1.layers.dense(input_data, units=emb_dim, kernel_initializer=glorot_uniform,
/home/federico/Documents/projects/rec-sys/RecSys-2023-polimi/Recommenders/GANMF/GANMF.py:67: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  decoding = tf.compat.v1.layers.dense(encoding, units=self.num_items, kernel_initializer=glorot_uniform,


Instructions for updating:
Use `tf.global_variables_initializer` instead.


2023-12-09 11:37:15.501019: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-09 11:37:15.544087: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-09 11:37:15.544568: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

51

In [14]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2136 (16.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10502 (100.0%) in 8.53 sec. Users per second: 1231


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.061284                 0.106942  0.090467  0.029535    0.050349   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.191518  0.097177  0.073069  0.394782      0.236527  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.830986          0.328058    0.830986       0.006774   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            7.600081                   0.991679             0.019438   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [8]:
# 67 epochs: [MAP: 0.0302020, MAP_MIN_DEN: 0.0510849]

<12638x22222 sparse matrix of type '<class 'numpy.float64'>'
	with 382984 stored elements in Compressed Sparse Row format>

In [9]:
import numpy as np
import scipy.sparse as sps

from Evaluation.Evaluator import EvaluatorHoldout

profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)
sorted_users = np.argsort(profile_length)

block_size = int(len(profile_length)*0.2)

MAP_recommender_per_group = {}
cutoff = 10

for group_id in range(0, 5):
    
    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id, 
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))
    
    
    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    users_not_in_group = sorted_users[users_not_in_group_flag]
    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)
    label = "GANMF"
    result_df, _ = evaluator_test.evaluateRecommender(final)
    if label in MAP_recommender_per_group:
        MAP_recommender_per_group[label].append(result_df.loc[cutoff]["MAP"])
    else:
        MAP_recommender_per_group[label] = [result_df.loc[cutoff]["MAP"]]
        

Group 0, #users in group 2527, average p.len 1.64, median 2.0, min 0, max 3
EvaluatorHoldout: Ignoring 2195 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10111 Users
EvaluatorHoldout: Processed 1064 (100.0%) in 1.20 sec. Users per second: 889
Group 1, #users in group 2527, average p.len 6.21, median 6.0, min 3, max 10
EvaluatorHoldout: Ignoring 2195 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10111 Users
EvaluatorHoldout: Processed 1911 (100.0%) in 1.38 sec. Users per second: 1382
Group 2, #users in group 2527, average p.len 14.75, median 14.0, min 10, max 21
EvaluatorHoldout: Ignoring 2195 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10111 Users
EvaluatorHoldout: Processed 2414 (100.0%) in 1.74 sec. Users per second: 1391
Group 3, #users in group 2527, average p.len 31.69, median 31.0, min 21, max 46
EvaluatorHoldout: Ignoring 2195 (17.4%) Users that have less than 1 test intera

In [10]:
MAP_recommender_per_group

{'GANMF': [0.005550147690655204,
  0.012661824192436465,
  0.01964455030838631,
  0.03426900171056273,
  0.05848074132700171]}